In [38]:
# We set the backend to TensorFlow. The code works with
# both `tensorflow` and `torch`. It does not work with JAX
# due to the behavior of `jax.numpy.tile` in a jit scope
# (used in `TransformerDecoder.get_causal_attention_mask()`:
# `tile` in JAX does not support a dynamic `reps` argument.
# You can make the code work in JAX by wrapping the
# inside of the `get_causal_attention_mask` method in
# a decorator to prevent jit compilation:
# `with jax.ensure_compile_time_eval():`.
import os

os.environ["KERAS_BACKEND"] = "tensorflow"


import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import tensorflow_datasets.public_api as tfds

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

In [39]:
text_pairs = []

with open("data.tsv", "r", encoding="utf-8") as f:
    for line in f:
        # Strip any extra whitespace and split the line by tabs
        fields = line.strip().split("\t")
        # Ensure the line has at least 4 fields (indexes 0 to 3)
        if len(fields) < 4:
            continue
        french = fields[1]  # second column
        portuguese = "[start] " + fields[3] + " [end]"  # fourth column
        text_pairs.append((french, portuguese))

# Print the first pair to verify
print(text_pairs[0])


('Je ne supporte pas ce type.', '[start] Eu não suporto esse tipo. [end]')


In [40]:
for _ in range(5):
    print(random.choice(text_pairs))

('Je veux perdre quelques kilos.', '[start] Eu quero perder alguns quilos. [end]')
("Nourrissez votre esprit d'informations importantes.", '[start] Alimente a sua mente com informações importantes. [end]')
('Les personnes fortes transpirent généralement beaucoup.', '[start] Pessoas gordas, geralmente, suam bastante. [end]')
("C'est un ouvrage traitant de l'Angleterre.", '[start] Essa é uma obra que trata da Inglaterra. [end]')
("Vous n'avez pas pu ouvrir la porte parce qu'elle était verrouillée de l'intérieur.", '[start] Você não conseguiu abrir a porta porque ela estava trancada por dentro. [end]')


In [41]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

33030 total pairs
23122 training pairs
4954 validation pairs
4954 test pairs


In [42]:
strip_chars = string.punctuation + "«" + "»"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 25000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


french_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
portuguese_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_french_texts = [pair[0] for pair in train_pairs]
train_portuguese_texts = [pair[1] for pair in train_pairs]
french_vectorization.adapt(train_french_texts)
portuguese_vectorization.adapt(train_portuguese_texts)

In [43]:
def format_dataset(french, portuguese):
    french = french_vectorization(french)
    portuguese = portuguese_vectorization(portuguese)
    return (
        {
            "encoder_inputs": french,
            "decoder_inputs": portuguese[:, :-1],
        },
        portuguese[:, 1:],
    )


def make_dataset(pairs):
    french_texts, portuguese_texts = zip(*pairs)
    french_texts = list(french_texts)
    portuguese_texts = list(portuguese_texts)
    dataset = tf_data.Dataset.from_tensor_slices((french_texts, portuguese_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [44]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


2025-02-08 01:16:49.352537: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [45]:
print(train_french_texts[:5])
print(train_portuguese_texts[:5])

["C'est une information intéressante.", "Peu importe si c'est bon, faisons le quand même.", 'Interlingua a cinq sœurs plus âgées.', "J'ai confessé mon péché.", "Ne te fais pas de souci, je vais t'aider."]
['[start] É uma informação interessante. [end]', '[start] Pouco importa se é bom, façamos mesmo assim. [end]', '[start] Interlíngua tem cinco irmãs mais velhas. [end]', '[start] Eu confessei o meu pecado. [end]', '[start] Não se preocupe, eu vou te ajudar. [end]']


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Parameters
sequence_length = 20  # Maximum number of tokens per sequence
vocab_size = 15000    # Total number of tokens in your vocabulary
embed_dim = 128       # Dimension of each token embedding

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        # Get token embeddings: shape (batch_size, sequence_length, embed_dim)
        embedded_tokens = self.token_embeddings(inputs)
        # Create a constant tensor for positions: shape (sequence_length,)
        positions = tf.range(start=0, limit=self.sequence_length, delta=1)
        # Get positional embeddings: shape (sequence_length, embed_dim)
        embedded_positions = self.position_embeddings(positions)
        # Expand dims to (1, sequence_length, embed_dim) so that it broadcasts over the batch
        embedded_positions = tf.expand_dims(embedded_positions, axis=0)
        # Return the sum of token and positional embeddings
        return embedded_tokens + embedded_positions


    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

# Build the encoder using the custom embedding layer and an LSTM
encoder_inputs = Input(shape=(None,), dtype="int64", name="encoder_inputs")
# Use our custom PositionalEmbedding layer
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# Pass the embeddings to an LSTM layer
encoder_lstm = layers.LSTM(32, return_state=True, name='encoder_lstm')
lstm_outputs, state_h, state_c = encoder_lstm(x)
encoder_states = [state_h, state_c]

# Define a simple decoder
decoder_inputs = Input(shape=(None,), dtype="int64", name="decoder_inputs")
decoder_embeddings = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
decoder_lstm = layers.LSTM(32, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)
decoder_dense = layers.Dense(vocab_size, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Build the final model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='seq2seq_model')

In [59]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "seq2seq_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, 20, 128)   │  1,922,560 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, 20, 128)   │  1,922,560 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 32),      │     20,608 │ positional_embed… │
│                     │ (None, 32),       │            │                   │
│                     │ (None, 32)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, 20, 32),  │     20,608 │ positional_embed… │
│                     │ (None, 32),       │            │ encoder_lstm[0][… │
│                     │ (None, 32)]       │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, 20, 15000) │    495,000 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,381,336 (16.71 MB)

 Trainable params: 4,381,336 (16.71 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# =============================================================================
# Prepare arrays for training and testing from the raw texts
# =============================================================================
# --- For training data ---
# Convert training texts into arrays using the vectorization layers
encoder_input_data = french_vectorization(np.array(train_french_texts)).numpy()
portuguese_tokenized = portuguese_vectorization(np.array(train_portuguese_texts)).numpy()
decoder_input_data = portuguese_tokenized[:, :-1]
decoder_target_data = portuguese_tokenized[:, 1:]

# --- For test data ---
test_french_texts = [pair[0] for pair in test_pairs]
test_portuguese_texts = [pair[1] for pair in test_pairs]
encoder_input_test = french_vectorization(np.array(test_french_texts)).numpy()
portuguese_tokenized_test = portuguese_vectorization(np.array(test_portuguese_texts)).numpy()
decoder_input_test = portuguese_tokenized_test[:, :-1]
decoder_target_test = portuguese_tokenized_test[:, 1:]

# =============================================================================
# Finally, call the training/testing function with the updated variables.
# =============================================================================
# (Assuming that ktrain_test is a function you have defined or imported
#  that takes the model and these data arrays.)
train_test(model,
            [encoder_input_data, decoder_input_data],
            decoder_target_data,
            [encoder_input_test, decoder_input_test],
            decoder_target_test,
            epochs=5,
            batch_size=64,
            patience=3,
            verbose=2)

NameError: name 'ktrain_test' is not defined